In [12]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

data = pd.read_parquet(Path("data") / "train.parquet")
external_data = pd.read_csv('submissions/external_data/external_data.csv')

def reinit():
    return(pd.read_parquet(Path("data") / "train.parquet"))

In [13]:
df = data.copy()
exd = external_data.copy()

In [14]:
def _encode_dates(X):
    X = X.copy()  # modify a copy of X
    # Encode the date information from the DateOfDeparture columns
    X.loc[:, "year"] = X["date"].dt.year
    X.loc[:, "month"] = X["date"].dt.month
    X.loc[:, "day"] = X["date"].dt.day
    X.loc[:, "weekday"] = X["date"].dt.weekday
    X.loc[:, "hour"] = X["date"].dt.hour

    # Finally we can drop the original columns from the dataframe
    return X.drop(columns=["date"])

In [15]:
def _cyclic_hours(X):
    X = X.copy()# modify a copy of X
    # Encode the date information from the DateOfDeparture columns
    X.loc[:, "hour_c"] = np.cos(np.pi/12 * X["hour"])
    X.loc[:, "hour_s"] = np.sin(np.pi/12 * X["hour"])
    # a way to show the cyclical ways of the hours
    return(X.drop(columns="hour"))

In [21]:
def merge_int_ex(X, external):
    """ Join X and Y on the same time period, with periods within external"""
    external = external.copy()
    X = X.copy()
    
    X.loc[:, "year"] = X["date"].dt.year
    X.loc[:, "month"] = X["date"].dt.month
    X.loc[:, "day"] = X["date"].dt.day
    X.loc[:, "time_period"] = X["date"].dt.hour //3

    external['date'] = pd.to_datetime(exd['date'])
    external.loc[:, "year"] = external["date"].dt.year
    external.loc[:, "month"] = external["date"].dt.month
    external.loc[:, "day"] = external["date"].dt.day
    external.loc[:, "time_period"] = external["date"].dt.hour //3
    external.drop(columns="date", inplace=True)

    u = X.set_index(['year', 'month', 'day', 'time_period'])
    v = external.set_index(['year', 'month', 'day', 'time_period'])
    w = u.join(v).reset_index()
    w.drop(columns=['year', 'month', 'day'], inplace=True)

    return(w)

## External data to use

In [19]:
# Colones à garder pour la fusion dans la external data. Critères =  
#   - les données logiques sont inclues dedans
#   - tous ces indices procurent au plus 0.4% de cellules vides (correspond à 10 cellules) 
#     et ce sont les seules

good_col = [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 15, 20, 23, 31, 32, 33, 38, 39]

exd = exd.iloc[:, good_col].copy()

## Internal data to use

In [18]:
df.drop(columns=["counter_name", "site_name", "bike_count", "counter_installation_date", 'counter_technical_id', 'coordinates'],inplace=True)

## Encodage de la donnée

internal :

counter : 1 hot encoder \
Site : one hot encoder ? ou evtl le drop au profit de la localisation\
installation date : osef normalement : n'a d'impact que sur le début (les gens découvrent la nouvelle station et adaptent leurs habitudes) ---- a rajouter dans un second temps.\
technical instl : osef \
bike count : osef, ou sinon faire du rev eng pour la target. \
year - month - day : commencer par un truc tel quel. on pourra rajouter les saison après pour marquer les séparations. \
hour : codé sur cos et sin \

external : laissé tel quel pour l'instant, avec du engeeniring plus tard. \



# Dataset

In [22]:
union = merge_int_ex(df, exd)

In [23]:
union.isna().sum()

time_period          0
counter_id           0
site_id              0
date                 0
latitude             0
longitude            0
log_bike_count       0
numer_sta          666
pmer               666
tend               666
cod_tend           666
dd                 666
ff                 666
t                  666
td                 666
u                  666
vv                 666
ww                 666
w1                1662
w2                2154
nbas              1500
pres               666
tend24            1998
raf10             1998
rafper             666
per                666
rr1               1506
rr3               1170
dtype: int64

In [30]:
n0= union.shape[0]
df = union.copy()
df.dropna(axis='index', inplace=True)
n1=df.shape[0]

info_loss= (1- n1/n0)*100

print(f"by removing all the na values, we lose {info_loss: .2f}% of the info of the total dataset.")

by removing all the na values, we lose  0.77% of the info of the total dataset.


Goal : find which columns containing NaN we removed and we could put back in, to find the best trade off between a lot of features, and less information.